<a href="https://colab.research.google.com/github/SeenaKhosravi/NASS/blob/main/NASS_Analysis_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required Python libraries
!pip install rpy2 matplotlib pandas

# Load rpy2 to run R code in the notebook
%load_ext rpy2.ipython

In [3]:
### Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
%%R

#load data
library(data.table)

# 1. Check if file exists:
file_path <- "/content/drive/My Drive/NASS_2020_all.csv"
if (file.exists(file_path)) {
  print("File found!")
} else {
  stop("File not found. Exiting.")
}

# 2. Attempt to load using fread:
NASS_2020_all <- fread(file_path)

# 3. Verify data structure:
str(NASS_2020_all)
colnames(NASS_2020_all)

Classes ‘data.table’ and 'data.frame':	7828310 obs. of  675 variables:
 $ KEY_NASS           : int  10000001 10000002 10000003 10000004 10000005 10000006 10000007 10000008 10000009 10000010 ...
 $ HOSP_NASS          : int  10446 10031 10145 10143 10326 10295 10202 10092 10357 10402 ...
 $ HOSP_TEACH         : int  1 1 1 1 1 1 1 1 1 1 ...
 $ HOSP_LOCATION      : int  1 1 1 1 1 1 1 1 1 1 ...
 $ HOSP_LOCTEACH      : int  3 3 3 3 3 3 3 3 3 3 ...
 $ HOSP_REGION        : int  1 1 1 1 1 1 1 1 1 1 ...
 $ HOSP_BEDSIZE_CAT   : int  1 3 3 3 3 3 3 3 2 2 ...
 $ DISCWT             : num  1.27 1.29 1.29 1.14 1.14 ...
 $ NASS_STRATUM       : int  40 58 58 33 33 58 33 33 40 40 ...
 $ N_DISC_U           : int  325943 381183 381183 658010 658010 381183 658010 658010 325943 325943 ...
 $ N_HOSP_U           : int  137 100 100 75 75 100 75 75 137 137 ...
 $ S_DISC_U           : int  256022 295908 295908 574998 574998 295908 574998 574998 256022 256022 ...
 $ S_HOSP_U           : int  105 79 79 63 63 79 63 6

In [ ]:
%%R
### Generate Overview Graphs
